In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!pip install -qr requirements.txt # install dependencies
!pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 10876, done.
remote: Total 10876 (delta 0), reused 0 (delta 0), pack-reused 10876
Receiving objects: 100% (10876/10876), 11.02 MiB | 24.58 MiB/s, done.
Resolving deltas: 100% (7510/7510), done.
/content/yolov5
     |████████████████████████████████| 596 kB 5.4 MB/s 
     |████████████████████████████████| 1.1 MB 7.7 MB/s 
     |████████████████████████████████| 178 kB 46.8 MB/s 
     |████████████████████████████████| 145 kB 45.5 MB/s 
     |████████████████████████████████| 138 kB 50.8 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 

In [ ]:
!unzip '/content/gdrive/MyDrive/custom_dataset.zip'  > /dev/null -d '/content/yolov5/custom_dataset'

In [ ]:
import json
import json
import os
from pathlib import Path

import requests
import yaml
from PIL import Image
from tqdm import tqdm

In [ ]:
def make_txt(file):
  names = []
  file = Path(file)


  with open(file) as f:  #load json
        json_data = json.load(f) 


  for data in json_data: 
      if data == 'ObjectInfo':
        lxtl, lytl, lxbr, lybr = json_data[data]['BoundingBox']['Leye']['Position']
        rxtl, rytl, rxbr, rybr = json_data[data]['BoundingBox']['Reye']['Position']
        # xtl, ytl, xbr, ybr : x상, y상, x하, y하

      if data == 'FileInfo':
        file_width = json_data[data]['Width']
        file_height = json_data[data]['Height']

  lxtl=float(lxtl)
  lytl=float(lytl)
  lxbr=float(lxbr)
  lybr=float(lybr)
  rxtl=float(rxtl)
  rytl=float(rytl)
  rxbr=float(rxbr)
  rybr=float(rybr)
  file_width=float(file_width)
  file_height=float(file_height)
  


  # 좌표 구하기
  l_width = lxbr - lxtl
  l_height = lybr - lytl
  l_center_x = (lxbr + lxtl)/2
  l_center_y = (lybr + lytl)/2

  r_width = rxbr - rxtl
  r_height = rybr - rytl
  r_center_x = (rxbr + rxtl)/2
  r_center_y = (rybr + rytl)/2

  f_dw = 1./file_width
  f_dh = 1./file_height

  l_x = l_center_x * f_dw
  l_y = l_center_y * f_dh
  l_w = l_width * f_dw
  l_h = l_height * f_dh

  r_x = r_center_x * f_dw
  r_y = r_center_y * f_dh
  r_w = r_width * f_dw
  r_h = r_height * f_dh

  l_box = ['0', str(l_x), str(l_y), str(l_w), str(l_h)]
  r_box = ['1', str(r_x), str(r_y), str(r_w), str(r_h)]

  return l_box, r_box

In [ ]:
make_txt('/content/yolov5/custom_dataset/test/label/R_071_30_M_01_M0_G1_C0_01.json')

(['0',
  '0.39381249999999995',
  '0.51410546875',
  '0.06968055555555558',
  '0.015226562500000007'],
 ['1',
  '0.5667152777777777',
  '0.49976171875',
  '0.07390277777777775',
  '0.015117187500000018'])

In [ ]:
path_dir='/content/yolov5/custom_dataset/test/label'
file_list = os.listdir(path_dir)

for i in range(len(file_list)):
  file_name = path_dir + '/' + file_list[i]
  if os.path.splitext(file_name)[1] == '.json':
    l_b, r_b = make_txt(file_name)
    new_file = file_name.replace('.json','')
    f = open(new_file+'.txt','w')
    f.write(' '.join(l_b))
    f.write('\n')
    f.write(' '.join(r_b))
    f.close()

In [ ]:
path_dir='/content/yolov5/custom_dataset/train/label'
file_list = os.listdir(path_dir)

for i in range(len(file_list)):
  file_name = path_dir + '/' + file_list[i]
  if os.path.splitext(file_name)[1] == '.json':
    l_b, r_b = make_txt(file_name)
    new_file = file_name.replace('.json','')
    f = open(new_file+'.txt','w')
    f.write(' '.join(l_b))
    f.write('\n')
    f.write(' '.join(r_b))
    f.close()

In [ ]:
path_dir='/content/yolov5/custom_dataset/valid/label'
file_list = os.listdir(path_dir)

for i in range(len(file_list)):
  file_name = path_dir + '/' + file_list[i]
  if os.path.splitext(file_name)[1] == '.json':
    l_b, r_b = make_txt(file_name)
    new_file = file_name.replace('.json','')
    f = open(new_file+'.txt','w')
    f.write(' '.join(l_b))
    f.write('\n')
    f.write(' '.join(r_b))
    f.close()

In [ ]:
 from glob import glob

In [ ]:
[os.remove(f) for f in glob("/content/yolov5/custom_dataset/test/label/*.json")]
[os.remove(f) for f in glob("/content/yolov5/custom_dataset/train/label/*.json")]
[os.remove(f) for f in glob("/content/yolov5/custom_dataset/valid/label/*.json")]

In [22]:
import yaml

make_yaml = {
             "train" : "/content/yolov5/custom_dataset/train",
             "val": "/content/yolov5/custom_dataset/valid",
            "test": "/content/yolov5/custom_dataset/test",
             "nc" : 2,
             "names": ["left_eye", "right_eye"]
}
with open('/content/yolov5/data/custom_dataset.yaml', 'w') as outfile:
    yaml.dump(make_yaml, outfile, default_flow_style=False)

In [24]:
!python train.py --img 640 --batch 16 --epochs 3 --data /content/yolov5/data/custom_dataset.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/yolov5/data/custom_dataset.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-255-gca0a007 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0

In [25]:
!python /content/yolov5/export.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --batch 16

export: data=data/coco128.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], imgsz=[640], batch_size=16, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'onnx']
YOLOv5 🚀 v6.0-255-gca0a007 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from /content/yolov5/runs/train/exp/weights/best.pt with output shape (16, 25200, 7) (14.4 MB)

TorchScript: starting export with torch 1.10.0+cu111...
TorchScript: export success, saved as /content/yolov5/runs/train/exp/weights/best.torchscript (28.5 MB)
requirements: onnx not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per ['onnx']
requirements: ⚠️ Restart runtime or rerun command for upd

In [26]:
pip install onnx_tf

     |████████████████████████████████| 222 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 41.3 MB/s 


In [36]:
!python /content/yolov5/models/tf.py --weights /content/yolov5/runs/train/exp/weights/best.pt

tf: weights=/content/yolov5/runs/train/exp/weights/best.pt, imgsz=[640, 640], batch_size=1, dynamic=False
Model Summary: 270 layers, 7025023 parameters, 0 gradients, 15.9 GFLOPs
2022-02-15 09:15:27.789310: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    115712  models.common.C3                

In [37]:
!python export.py --weights /content/yolov5/runs/train/exp/weights/best.pt --include tflite

export: data=data/coco128.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tflite']
YOLOv5 🚀 v6.0-255-gca0a007 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from /content/yolov5/runs/train/exp/weights/best.pt with output shape (1, 25200, 7) (14.4 MB)

TensorFlow SavedModel: starting export with tensorflow 2.7.0...

                 from  n    params  module                                  arguments                     
2022-02-15 09:17:06.550786: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
  0                -1

In [38]:
from google.colab import files

files.download('/content/yolov5/runs/train/exp/weights/best-fp16.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
import onnx
from onnx_tf.backend import prepare

base_model = onnx.load('/content/yolov5/runs/train/exp/weights/best.onnx')
to_tf = prepare(base_model)
to_tf.export_graph("/content/yolov5/runs/train/exp/weights/best.pb")

TypeError: ignored

In [31]:
# pb to tflite
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

graph_def_file = "/content/yolov5/runs/train/exp/weights/best.pb/saved_model.pb"
input_arrays = ["images"]
output_arrays = ["output"]
tflite_file = "/content/yolov5/runs/train/exp/weights/best.tflite"

converter = tf.lite.TFLiteConverter.from_frozen_graph(graph_def_file, input_arrays, output_arrays)

tflite_model = converter.convert()

open(tflite_file, "wb").write(tflite_model)

Ignore 'tcmalloc: large alloc' warnings.


UnicodeDecodeError: ignored